In [ ]:
import glob
import os
import re

In [ ]:
seed_no = 4

In [ ]:
dihedral = ['8345'] # list for combining torsions of the same type as needed
fragment = ['4']

In [ ]:
# process mol2 files to rename atom types as gaff2 sometimes assigns weird atom types
# also changing the atom type of the CA from c3 (gaff2) to CX 

def process_mol2_file(input_file, output_file):
    """
    Processes a single .mol2 file to rename specific atom types while preserving formatting.

    Args:
        input_file (str): Path to the input .mol2 file.
        output_file (str): Path to save the modified .mol2 file.
    """
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    updated_lines = []
    in_atom_section = False

    for line in lines:
        if line.strip() == "@<TRIPOS>ATOM":
            in_atom_section = True
        elif line.strip() == "@<TRIPOS>BOND":
            in_atom_section = False

        if in_atom_section and line.strip() and not line.startswith("@<TRIPOS>"):
            # example that replaces the atom type of C2 (the CA) to CX
            if re.search(r'\sC3\s', line):
                line = re.sub(r'(\s)c3(\s)', r'\1CX\2', line)
            line = re.sub(r'(\s)os(\s)', r'\1o \2', line)
            line = re.sub(r'(\s)n7(\s)', r'\1ns\2', line)
            line = re.sub(r'(\s)h1(\s)', r'\1hc\2', line)

        updated_lines.append(line)

    with open(output_file, 'w') as outfile:
        outfile.writelines(updated_lines)


def batch_process_mol2_files(input_dir):
    """
    Batch processes all .mol2 files in a directory.

    Args:
        input_dir (str): Directory containing input .mol2 files.
    """
    for filename in os.listdir(input_dir):
        if filename.endswith(".mol2"):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(input_dir, f"{os.path.splitext(filename)[0]}_atomtypes_renamed.mol2")
            print(f"Processing {input_file} -> {output_file}")
            process_mol2_file(input_file, output_file)


input_directory = "../profiles_torsions/optimised_geometries/fragment4/torsion2345/" 
batch_process_mol2_files(input_directory) # process all mol2 files from the QM scans

In [ ]:
file_list = []

for frag in fragment:
    for dih in dihedral:
        file_pattern = '../profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf*_resp_charges_atomtypes_renamed.mol2' % (frag, dih, dih, seed_no)

        file_list.extend(glob.glob(file_pattern))

file_list

In [ ]:
# create prmtop and inpcrd files for all mol2 files

for fle in file_list:
    tleapinput = open("tleap.in", "w")
    tleapinput.write(f"mol  = loadmol2 {fle} \n")
    tleapinput.write("loadamberparams ../frcmod_files/R8-S5_alanine/R8-S5_alanine_gaff2.frcmod \n") # use frcmod previously made
    # this frcmod will need to be updated after each torsion fit, so the next batch of mol2 are parameterised with the updated file
    tleapinput.write(f"saveamberparm mol {fle}.prmtop {fle}.inpcrd  \n")
    tleapinput.write("quit")
    tleapinput.close()

    os.system("tleap -f tleap.in")
